In [6]:
# 导入同花顺包和登录同花顺
import datetime as dt
import pandas as pd
from numpy import *
from iFinDPy import *
THS_iFinDLogin("gdzq6888", "199212")

0

In [51]:
from datetime import datetime
from calendar import monthrange
import numpy as np

In [7]:
def split(list_a, chunk_size):
    for i in range(0, len(list_a), chunk_size):
        yield list_a[i:i + chunk_size]

# 多头alpha

## 个股截面离散度

In [28]:
# 输入日期格式为’年-月‘
def individual_stock_dispersion(time):
    date = datetime.strptime(time, '%Y-%m')
    days_in_month = monthrange(date.year, date.month)[1]
    month_end = date + timedelta(days=days_in_month - date.day)
    month_end_10years = date.replace(year=date.year-10)
    month_end = str(month_end)[:10]
    month_end_10years = str(month_end_10years)[:10]
    # 获取中证800指数成分
    data = THS_DP('index_weight',month_end+';000906.SH','date:Y,thscode:Y,security_name:Y,weight:Y').data
    # 提取股票代码
    stock_code = list(data['thscode'])
    # 将股票代码分为4份
    stock_code2 = list(split(stock_code,200))
    #获得成分股每日价格
    data = pd.DataFrame()
    for i in stock_code2:
        a = THS_HQ(i,'close','',month_end_10years,month_end).data
        data=pd.concat([data,a])
    # 加入月，日
    data['year'] = data['time'].apply(lambda x: x.split('-')[0])
    data['month'] = data['time'].apply(lambda x: x.split('-')[1])
    # 计算回报
    data['lag'] = data.groupby(['thscode'])['close'].shift(1)
    data['return'] = (data['close']-data['lag'])/data['lag']
    data = data.dropna()

    # 计算日标准差
    std = data.groupby(['time'])['return'].std().reset_index()
    std = std.rename(columns={'return':'daily std'})
    std['month'] = std['time'].str[:7]

    # 得到离散度
    std_monthly = std.groupby('month')['daily std'].mean().reset_index()
    std_monthly = std_monthly.rename(columns={'daily std':'monthly std'})

    # 计算高/低离散度
    dispersion = list(std_monthly['monthly std'])[-1]/mean(list(std_monthly['monthly std']))

    return dispersion


In [30]:
individual_stock_dispersion('2023-01')

0.7584548441638147

## 行业截面离散度

In [42]:
def industry_dispersion(time):
    date = datetime.strptime(time, '%Y-%m')
    days_in_month = monthrange(date.year, date.month)[1]
    month_end = date + timedelta(days=days_in_month - date.day)
    month_end_10years = date.replace(year=date.year-10)
    month_end = str(month_end)[:10]
    month_end_10years = str(month_end_10years)[:10]

    # 获取一级行业指数回报
    data=THS_HQ('CI005001.CI,CI005002.CI,CI005003.CI,CI005004.CI,CI005005.CI,CI005006.CI,CI005007.CI,CI005008.CI,CI005009.CI,CI005010.CI,CI005011.CI,CI005012.CI,CI005013.CI,CI005014.CI,CI005015.CI,CI005016.CI,CI005017.CI,CI005018.CI,CI005019.CI,CI005020.CI,CI005021.CI,CI005022.CI,CI005023.CI,CI005024.CI,CI005025.CI,CI005026.CI,CI005027.CI,CI005028.CI,CI005029.CI,CI005030.CI','close','',month_end_10years,month_end).data

    # 计算回报
    data['lag'] = data.groupby(['thscode'])['close'].shift(1)
    data['return'] = (data['close']-data['lag'])/data['lag']
    data = data.dropna()

    # 计算日标准差
    std = data.groupby(['time'])['return'].std().reset_index()
    std = std.rename(columns={'return':'daily std'})
    std['month'] = std['time'].str[:7]

    # 得到离散度
    std_monthly = std.groupby('month')['daily std'].mean().reset_index()
    std_monthly = std_monthly.rename(columns={'daily std':'monthly std'})

    # 计算高/低离散度
    dispersion = list(std_monthly['monthly std'])[-1]/mean(list(std_monthly['monthly std']))

    return  dispersion


In [43]:
industry_dispersion('2023-01')

0.9762148769812952

## 个股相关性

In [55]:
# 输入日期格式为’年-月‘
def stock_correlation(time):
    date = datetime.strptime(time, '%Y-%m')
    days_in_month = monthrange(date.year, date.month)[1]
    month_end = date + timedelta(days=days_in_month - date.day)
    month_end_10years = date.replace(year=date.year-10)
    month_end = str(month_end)[:10]
    month_end_10years = str(month_end_10years)[:10]
    # 获取中证800指数成分
    data = THS_DP('index_weight',month_end+';000906.SH','date:Y,thscode:Y,security_name:Y,weight:Y').data
    # 提取股票代码
    stock_code = list(data['thscode'])
    # 将股票代码分为4份
    stock_code2 = list(split(stock_code,200))
    #获得成分股每日价格
    data = pd.DataFrame()
    for i in stock_code2:
        a = THS_HQ(i,'close','',month_end_10years,month_end).data
        data=pd.concat([data,a])

    # 计算回报
    data['lag'] = data.groupby(['thscode'])['close'].shift(1)
    data['return'] = (data['close']-data['lag'])/data['lag']
    data = data.dropna()

    return data



In [56]:
data = stock_correlation('2023-01')

In [57]:
data

,time,thscode,close,lag,return
1,2013-01-07,600519.SH,204.29,206.94,-0.012806
2,2013-01-08,600519.SH,210.84,204.29,0.032062
3,2013-01-09,600519.SH,216.45,210.84,0.026608
4,2013-01-10,600519.SH,214.50,216.45,-0.009009
5,2013-01-11,600519.SH,211.26,214.50,-0.015105
...,...,...,...,...,...
392740,2023-01-18,002901.SZ,36.80,36.91,-0.002980
392741,2023-01-19,002901.SZ,37.15,36.80,0.009511
392742,2023-01-20,002901.SZ,36.91,37.15,-0.006460
392743,2023-01-30,002901.SZ,37.24,36.91,0.008941


In [58]:
grouped_data = data.groupby('time')

In [52]:
# sample data
data = {'group': ['A', 'B', 'A', 'B', 'B', 'C'],
        'var1': [1, 2, 3, 4, 5, 6],
        'var2': [10, 20, 30, 40, 50, 60]}
df = pd.DataFrame(data)

# group by 'group' column
grouped = df.groupby('group')

# calculate average correlation between each group
correlations = []
for name, group in grouped:
    var1 = group['var1']
    var2 = group['var2']
    correlations.append(np.corrcoef(var1, var2)[0][1])

average_correlation = np.mean(correlations)

D:\Anaconda\lib\site-packages\numpy\lib\function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
D:\Anaconda\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
D:\Anaconda\lib\site-packages\numpy\lib\function_base.py:2542: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


In [53]:
df

,group,var1,var2
0,A,1,10
1,B,2,20
2,A,3,30
3,B,4,40
4,B,5,50
5,C,6,60
